# Exercises Week 12 - Association Rule Mining

In [1]:
import numpy as np

# Local imports
import sys
sys.path.append('./utilities')
from load_data import load_market_basket

## Exercise 1.

The work you do in this exercise will be very useful also in your hand-in.

We learned the Apriori algorithm in class. Make your own implementation. 

We will use the anonymized real-world retail market basket data from: http://fimi.ua.ac.be/data/.

This data comes from an anonymous Belgian retail store, and was donated by Tom Brijs from Limburgs Universitair Centrum, Belgium. The original data contains 16,470 different items and 88,162 transactions. You may only work with the top-50 items in terms of occurrence frequency.

Your task is to:
1. Implement the Apriori algorithm.
2. Apply the Apriori algorithm on these data to find association rules with minimum support 0.05 and minimum confidence 0.7. Write down the rules in descending order of confidence.


In [2]:
# Load the retail data
transactions = load_market_basket()

def book_example():
    return [
        [1, 2, 4, 5],
        [2, 3, 5],
        [1, 2, 4, 5],
        [1, 2, 3, 5],
        [1, 2, 3, 4, 5], 
        [2, 3, 4],
    ]

def filter_transactions(T, k=50):
    """
        Keep only the top k items in the transactions.
        Remove transactions that become empty.
    """
    # Count occurences of each item
    counts = [0] * 16470
    for t in T:
        for i in t:
            counts[i] += 1

    # Sort and select top k
    counts = np.array(counts)
    order  = np.argsort(counts)[::-1] # reverse the sorted order

    indexes_to_keep = order[:k]       # Keep the top k items
    index_set = set(indexes_to_keep)  # Convert to python set for efficiency

    # Filter transactions
    T_new = [t_ for t_ in  [list(filter(lambda i: i in index_set, t)) for t in T]  if t_]
    return T_new

T = filter_transactions(transactions, k=50)
#T = book_example() # You can first use the very small dataset to test

In [65]:
# Tiny function for generating rules from tuples
# Ex: rule((1, 2), (5)) outputs "(1, 2) => (5)"
rule  = lambda lhs, rhs: "%s => %s" % (str(lhs), str(rhs)) # For generating rule strings
from itertools import chain, combinations

def compute_support(Ck, T, still_applicable=None):
    # counts = []
    # ### TODO Your code here
    # for i in Ck:
    #     count_i = 0
    #     for j in T:
    #         check =  all(item in j for item in i)
    #         if check:
    #             count_i += 1 
    #     counts.append(count_i / len(T))
    # ### TODO Your code here
    # return counts

    if still_applicable is None: still_applicable = [True]* len(T)

    counts = {}
    for row, t in enumerate(T):
        found_any = False
        for c in Ck:
            if set.issubset(set(c), t):
                if c in counts: counts[c] += 1
                else: counts[c] =1
                found_any = True

        still_applicable[row] = found_any
    return counts

def extend_prefix_tree(Ck_prev):
    Ck = set()
    ### TODO Your code here
    for itemset in Ck_prev:
        its1 = tuple(sorted(itemset))
        for itemset2 in Ck_prev:
            its2 = tuple(sorted(itemset2))
            if its1[:-1] == its2[:-1] and its1[-1] < its2[-1]: Ck.add(its1 + its2[-1:])

    to_remove = set()
    for c in Ck:
        for subset in combinations(c, len(c)-1):
            if not subset in Ck_prev:
                to_remove.add(c)
                break
    
    for c in to_remove:
        Ck.remove(c)
    ### TODO Your code here
    return Ck

def powerset(iterable):
    ### TODO Your code here
    # powerset = []
    # for i in iterable:
    #     for j in iterable:
    #         if i != j:
    #             intersection = list(set(i) | set(j))
    #             if len(intersection) > len(i) + 1:
    #                 continue
    #             else:
    #                 if intersection not in powerset:
    #                     powerset.append(intersection)
    # ### TODO Your code here
    # return powerset
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

def apriori_algorithm(T, support=0.05, min_confidence=0.7):
    """
        Apriori algorithm for mining association rules.
        Inputs:
            T:               A list of lists, each inner list will contiain integer-item-ids. 
                             Example: T = [[1, 2, 5], [2, 3, 4], [1, 6]]
            support:         The proportion of occurences needed to keep itemsets.
            min_confidence:  Minimum confidence for the algorithm to output the rule.
        
        Outputs:
            rules:           List of tuples [(rule:str, confidence:float), ... ]
                             Example: [("(1, 2) => (5)", 0.84), ("(3, 4) => (7)", 0.75)]
    """
    ### TODO Your code here
    # C1 = [[x] for x in set(chain(*T))]
    
    # while(len(C1) != 0):
    #     support_list = compute_support(C1, book_example())

    #     tmp = []
    #     for idx, i in enumerate(support_list):
    #         if i >= support:
    #             tmp.append(C1[idx])

    #     C1 = powerset(tmp)
    #     print("Ck ", C1)

    n = len(T)

    itemsets = {}
    C1 = set()
    for t in T:
        for t1 in t: C1.add((t1,))

    still_applicable = [True] * n
    Ck = C1

    k = 1

    while Ck:
        itemsets[k] = compute_support(Ck, T, still_applicable)
        Ck_copy = Ck.copy()
        for itemset in Ck:
            if itemsets[k][itemset] / n < support:
                del itemsets[k][itemset]
                Ck_copy.remove(itemset)
        
        Ck = extend_prefix_tree(Ck_copy)
        k += 1
    
    k = 2
    rules = []
    while k <= max(itemsets.keys()):
        for itemset in itemsets[k].keys():
            for rhs in powerset(itemset):
                remaining = set(itemset).difference(set(rhs))
                lhs = tuple(sorted(remaining))

                conf = itemsets[k][itemset] / itemsets[len(lhs)][lhs]
                if conf >= min_confidence:
                    rules.append((rule(lhs, rhs), conf))
        k += 1
    ### TODO Your code here
    return rules

Below you can try your algorithm. If you want, you could try changing the support and min_confidence to see what happens to the result.

In [66]:
rules = apriori_algorithm(T, support=0.05, min_confidence=0.7)
rules = sorted(rules, key=lambda x: x[1], reverse=True)

print("%-8s \t %s" % ("Conf.", "Rule"))
for r in rules:
    print("%7.4f%% \t %s" % r[::-1])

Conf.    	 Rule
 0.8168% 	 (41, 48) => (39,)
 0.7681% 	 (38, 48) => (39,)
 0.7637% 	 (41,) => (39,)


## Exercise 2.

We have learned how to mine frequent itemsets and association rules from a transaction database where each transaction consists of a simple set of items. You are asked to propose a framework for mining association rules from transaction data, in which each item in a transaction is associated with an integer number that counts how many times the items appears in the transaction. In a market basket, this count number indicates the number of copies of a product in a customer’s basket. For example, we do not only care whether a customer bought fish or not, but how many pieces of fish they bought.

1. Define (extend) the notion of an itemset and an association rule in the case of such data.
2. Describe an efficient algorithm that mines itemsets and association rules as defined in Exercise 2. Illustrate the pruning strategies used in your algorithm and explain how they relate to the Apriori principle.
3. Extend your implementation of the Apriori algorithm to handle this case.

In [67]:
### TODO your code here
# You can copy pase the code from above and adjust it

### TODO your code here

## Exercise 3

Consider a dataset of transactions $D$, and let $D'$ be a dataset derived from $D$ by independently deleting items from transactions in $D$. In particular, any item in any transaction in $D$ is deleted with probability $p$.

1. Given an itemset $S$, compute its expected support in $D'$ as a function of its support in $D$.
2. Compute the probability that an itemset $S$, which is frequent in $D$, is also frequent in $D'$, under the same minimum support threshold.
